<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Sujet-de-TP-:-modèle-de-langage-avec-un-RNN-&quot;Vanilla&quot;" data-toc-modified-id="Sujet-de-TP-:-modèle-de-langage-avec-un-RNN-&quot;Vanilla&quot;-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Sujet de TP : modèle de langage avec un RNN "Vanilla"</a></span></li><li><span><a href="#Préparation-des-données" data-toc-modified-id="Préparation-des-données-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Préparation des données</a></span></li><li><span><a href="#imports-et-accès-à-un-GPU" data-toc-modified-id="imports-et-accès-à-un-GPU-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>imports et accès à un GPU</a></span></li><li><span><a href="#Chargement-du-corpus" data-toc-modified-id="Chargement-du-corpus-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Chargement du corpus</a></span></li><li><span><a href="#Créer-la-classe-du-modèle" data-toc-modified-id="Créer-la-classe-du-modèle-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Créer la classe du modèle</a></span></li><li><span><a href="#Instancier-le-modèle" data-toc-modified-id="Instancier-le-modèle-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Instancier le modèle</a></span></li><li><span><a href="#Apprentissage" data-toc-modified-id="Apprentissage-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Apprentissage</a></span></li><li><span><a href="#Tester-le-modèle-sur-des-phrases" data-toc-modified-id="Tester-le-modèle-sur-des-phrases-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Tester le modèle sur des phrases</a></span></li></ul></div>

# Sujet de TP : modèle de langage avec un RNN "Vanilla"

Dans ce TP, nous allons créer un RNN simple (un "Vanilla RNN") qui prédit le mot qui suit un début de phrase.

Nous allons entraîner ce RNN sur un tout petit sous-ensemble de textes provenant du corpus Librivox French (https://librivox.org) qui regroupe des audiobooks.

Une fois entraîné, vous pouvez compléter un début de phrase en faisant des prédictions avec le modèle.

Nous utilisons les balises < unk > et < eos > pour les mots qui ne sont pas dans notre vocabulaire et pour indiquer une fin de phrase, respectivement.
    
Nous avons restreint le vocabulaire à la liste de mots apparaissant au moins 4 fois dans Librivox.

# Préparation des données

Nous vous fournissons un notebook tout prêt pour cela : generate_librivox_fr.ipynb

Ouvrez-le et lisez-le. Tentez de comprendre à quoi sert chaque objet et chaque cellule du notebook.

Exécutez chaque cellule. 

Quels fichiers ont été créés ? Que contiennent-ils ?

# imports et accès à un GPU

In [ ]:
# # Pour Google Colab
# import sys, os
# if 'google.colab' in sys.modules:
#     from google.colab import drive
#     drive.mount('/content/gdrive')
#     file_name = 'vrnn_demo.ipynb'
#     import subprocess
#     path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
#     print(path_to_file)
#     path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
#     os.chdir(path_to_file)
#     !pwd

In [43]:
import os
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import re

Avec ou sans GPU ?

Il est recommandé d'exécuter ce code sur GPU :<br> 
* Temps pour 1 epoch sur CPU : 153 sec ( 2.55 min)<br> 
* Temps pour 1 epoch sur GPU : 8.4 sec avec une GeForce GTX 1080 Ti <br>

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
device= torch.device("cuda")
#device= torch.device("cpu")
print(device, torch.cuda.get_device_name(device=None))

cuda GeForce GTX 1080


In [4]:
torch.cuda.empty_cache()

# Chargement du corpus


Vérifier que vous avez bien généré les données : 

In [5]:
path_data='../'

flag_train_data = os.path.isfile(path_data + 'librivox_fr/train_data.pt') 
flag_test_data = os.path.isfile(path_data + 'librivox_fr/test_data.pt') 

flag_idx2word = os.path.isfile(path_data + 'librivox_fr/idx2word.pt') 
flag_word2idx = os.path.isfile(path_data + 'librivox_fr/word2idx.pt') 

if flag_idx2word==False or flag_test_data==False or flag_train_data==False or flag_word2idx==False:
    print('Librivox_fr dataset manquant')


Charger train_data et test_data :

In [7]:
train_data  =  torch.load(path_data+'librivox_fr/train_data.pt')
test_data   =  torch.load(path_data+'librivox_fr/test_data.pt')

print(train_data.size())
print(test_data.size())

torch.Size([20542, 20])
torch.Size([74, 20])


Charger les dictionnaires idx2word et word2idx :

In [8]:
word2idx  =  torch.load(path_data + 'librivox_fr/word2idx.pt')
idx2word  =  torch.load(path_data + 'librivox_fr/idx2word.pt')

la première phrase du texte train_librivox_fr_50words_max_15200.txt est : 

"enfant si j’étais roi je donnerais l’empire et mon char et mon sceptre et mon peuple à genoux et ma couronne d’or et mes bains de porphyre et mes flottes à qui la mer ne peut suffire"

Comment est-elle stockée dans le tenseur train_data ?

Afficher les 38 premiers mots dans train_data qui correspondent à cette phrase jusqu'à < eos >. Où est stockée la deuxième phrase ?


In [25]:
seq_length = 38

phrase_index = 0
print("phrase index=", phrase_index)

# create a minibatch
minibatch_data =  train_data[:seq_length]
#     print(minibatch_data)
for i in range(seq_length):
    print("%d:%s"%(minibatch_data[i][phrase_index], idx2word[minibatch_data[i][phrase_index]]), end=" ")


phrase index= 0
1:enfant 2:si 3:j’étais 4:roi 5:je 6:donnerais 7:l’empire 8:et 9:mon 10:char 8:et 9:mon 11:sceptre 8:et 9:mon 12:peuple 13:à 14:genoux 8:et 15:ma 16:couronne 17:d’or 8:et 18:mes 19:bains 20:de 21:porphyre 8:et 18:mes 0:<unk> 13:à 22:qui 23:la 24:mer 25:ne 26:peut 27:suffire 28:<eos> 

En déduire comment les données sont présentées au futur modèle.

Rep : en ligne les phrases et en colonne les batch. C'est-à-dire que les mots des premières lignes du ligne sont stockées dans la première colonne jusqu'à atteindre 20542 mots, puis on passe à la deuxième colonne, etc. Il y aura la troisième dimension qui sera la taille des embeddings...

In [ ]:
20*20542

In [ ]:
max(torch.unique(train_data)), len((torch.unique(train_data)))

In [ ]:
max(torch.unique(test_data)), len((torch.unique(test_data)))

l'indice 9575 dépasse le tableau ---> bug dans test_data

Quelques constantes associées au dataset

In [22]:
bs = 20 # taille des batches : si modifiée, il faut regénérer train et test avec ce bs dans generate_librivox_fr
seq_length = 35 # taille des "phrases" à donner au réseau

# vocab_size = 17498 # if WORD_OCC_THRESHOLD == 1
vocab_size = 9574 # if WORD_OCC_THRESHOLD == 3

# Créer la classe du modèle

Compléter la définition du modèle à trois couches suivant, en indiquant les bonnes dimensions.

Compléter la définition de la fonction forward.

In [26]:
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size):
        
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding( vocab_size  , hidden_size  )
        self.layer2 = nn.RNN(       hidden_size , hidden_size  )
        self.layer3 = nn.Linear(    hidden_size , vocab_size   )

        
    def forward(self, word_seq, h_init ):
        
        g_seq               =   self.layer1( word_seq )  
        h_seq , h_final     =   self.layer2( g_seq , h_init )
        score_seq           =   self.layer3( h_seq )
        
        return score_seq,  h_final 


# Instancier le modèle

Écrire une fonction qui affiche le nombre de paramètres d'un réseau donné en argument (net)

In [30]:
def display_num_param(net):
    nb_param = 0
    for param in net.parameters():
        nb_param += param.numel()
    print('Nombre de paramètres du réseau : {} ({:.2f} millions)'.format(
        nb_param, nb_param/1e6)
         )



Choisir une hidden size de 150. Combien de paramètres contient le modèle?

In [31]:
hidden_size=150

net = three_layer_recurrent_net( hidden_size )

print(net)

display_num_param(net)

three_layer_recurrent_net(
  (layer1): Embedding(9574, 150)
  (layer2): RNN(150, 150)
  (layer3): Linear(in_features=150, out_features=9574, bias=True)
)
Nombre de paramètres du réseau : 2927074 (2.93 millions)


Envoyer le modèle sur le GPU (si vous utilisez un gpu)

In [32]:
net = net.to(device)

Initialiser les poids de la couche embedding et de la couche linéaire avec une distribution uniforme sur [-0.1, 0.1]

In [33]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

# Apprentissage

Définir la fonction de coût entropie croisée et les hyperparamètres suivants : 
* learning rate initial : my_lr=1
* taille des séquences : seq_length=35

In [34]:
criterion = nn.CrossEntropyLoss()

my_lr = 1.

seq_length = 35

Lors de l'apprentissage, pour éviter le phénomène d'explosion du gradient, nous allons utiliser une fonction qui normalise les valeurs du gradient

In [36]:
def normalize_gradient(net):

    grad_norm_sq=0

    for p in net.parameters():
        grad_norm_sq += p.grad.data.norm()**2

    grad_norm=math.sqrt(grad_norm_sq)
   
    if grad_norm<1e-4:
        net.zero_grad()
        print('norme du gradient proche de zéro')
    else:
        for p in net.parameters():
             p.grad.data.div_(grad_norm)

    return grad_norm



Voici une fonction qui évalue le réseau sur le jeu de test (non-utilisée car bug dans données pour l'instant)

In [35]:
# def eval_on_test_set():

#     running_loss=0
#     num_batches=0    
       
#     h = torch.zeros(1, bs, hidden_size)
    
#     h=h.to(device)

#     for count in range( 0 , 74-seq_length ,  seq_length) :
               
#         minibatch_data =  test_data[ count   : count+seq_length   ]
#         minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
#         minibatch_data=minibatch_data.to(device)
#         minibatch_label=minibatch_label.to(device)
                                  
#         scores, h  = net( minibatch_data, h )
        
#         minibatch_label =   minibatch_label.view(  bs*seq_length ) 
#         scores          =            scores.view(  bs*seq_length , vocab_size)
        
#         loss = criterion(  scores ,  minibatch_label )    
        
#         h=h.detach()
            
#         running_loss += loss.item()
#         num_batches += 1        
    
#     total_loss = running_loss/num_batches 
#     print('test: exp(loss) = ', math.exp(total_loss)  )
        

Compléter la boucle d'apprentissage aux endroits indiqués et entraîner le modèle sur 10 epochs

In [38]:
start=time.time()

for epoch in range(1):
    
    # garder le learning rate à 1 pour les 4 premières epochs, puis diviser par 1.1 à chaque epoch
    if epoch >= 4:
        my_lr = my_lr / 1.1
    
    # créer un nouvel optimizer et lui passer le learning rate actualisé.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # initialisation du coût et du nombre de batchs à chaque nouvelle epoch 
    running_loss=0
    num_batches=0    
    
    # initialiser h par un vecteur de zéros :
    h = torch.zeros(1, bs, hidden_size)

    # envoi au gpu    
    h=h.to(device)
    
    for count in range( 0 , 20542-seq_length ,  seq_length):
             
        # Mettre les gradients à zéro
        optimizer.zero_grad()
        
        # créer un minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1 : count+seq_length+1 ]        
        
        if count == 0:
            print('X:', minibatch_data.shape)
            print('Y:', minibatch_label.shape)
        
        # envoi au gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detacher h pour ne pas backpropager sur toutes les séquences depuis le début de l'epoch
        h=h.detach()
        # Dire à Pytorch de tracker les opérations sur h pour le minibatch courant
        h=h.requires_grad_()
                       
        # Passe forward
        scores, h  = net( minibatch_data, h )
        
        if count == 0:
            print('scores:', scores.shape)
            print('h:', h.shape)
        
        # Reshape les tenseurs scores et labels pour obtenir une longueur de bs*seq_length
        scores          =            scores.view(  bs*seq_length , vocab_size)  
        minibatch_label =   minibatch_label.view(  bs*seq_length )       
        
        # Calculer la loss moyenne
        loss = criterion(  scores ,  minibatch_label )
        
        # Passe backward pour calculer les gradients dL/dR, dL/dV et dL/dW
        loss.backward()

        # Normaliser les gradients et faire une itération de SGD : R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        normalize_gradient(net)
        optimizer.step()
        
        # Actualiser le coût par epoch et le nb de batches traités  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # calcul du coût sur tout le training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    
    # Estimer la performance sur le jeu de test (bug pour l'instant) 
    #     eval_on_test_set() 


X: torch.Size([35, 20])
Y: torch.Size([35, 20])
scores: torch.Size([35, 20, 9574])
h: torch.Size([1, 20, 150])

epoch= 0 	 time= 3.2571892738342285 	 lr= 1.0 	 exp(loss)= 717.5571866956979


Question : quelle est la taille des tenseurs suivants ?
        - minibatch_data, minibatch_label,
        - h, scores avant le reshape
        - h, scores après le reshape


# Tester le modèle sur des phrases

Voici une fonction qui prend une phrase et qui la convertit en tenseur exploitable pour le réseau

In [40]:
def sentence2vector_librivox_fr(sentence):
    words = sentence.split()
    x = torch.LongTensor(len(words),1)
    for idx, word in enumerate(words):
        word = re.sub("'", "_", word)
        if word not in word2idx:
            print('Vous avez entrer un mot hors-vocabulaire.')
            print('--> Enlever lettres majuscules et ponctuation')
            print("mot --> <unk> avec index 0")
            x[idx,0]=0            
        else:
            x[idx,0]=word2idx[word]
    return x


In [41]:
sentence1 = "on entendait vaguement au dehors les"

sentence2 = "hier je luttai de la sorte contre le grand"

sentence3 = "il connaissait la route et nous avons"

# ou bien créer votre propre phrase. Il ne fauit pas utiliser de majuscules ni de ponctuation.
# Chaque mot doit être dans le lexique.
sentence4= "il est beaucoup"

# Choisir le phrase ici : 
mysentence = sentence1

Convertir la phrase choisie et envoi au GPU

In [44]:
minibatch_data=sentence2vector_librivox_fr(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[ 366],
        [1383],
        [1884],
        [  72],
        [ 127],
        [  37]], device='cuda:0')


Définir un hidden state initial à zero, et exécuter le RNN sur la phrase

In [45]:
h = torch.zeros(1, 1, hidden_size)
h=h.to(device)

scores , h = net( minibatch_data , h )

Écrire une fonction show_next_word qui prend en entrée scores et qui affiche les 30 mots les plus probables prédits par le réseau.

Vous utiliserez la fonction torch.topk()
Aide : https://pytorch.org/docs/stable/torch.html?highlight=topk#torch.topk


In [47]:
def show_next_word(scores):
    num_word_display=30
    prob = F.softmax(scores,dim=2)
    p = prob[-1].squeeze()
    p, word_idx = torch.topk(p, num_word_display)

    for i,idx in enumerate(word_idx):
        percentage= p[i].item()*100
        word=  idx2word[idx.item()]
        print(  "{:.1f}%\t".format(percentage),  word ) 


Afficher la prédiction du prochain mot par le réseau

In [49]:
print(mysentence, '... \n')

show_next_word(scores)

on entendait vaguement au dehors les ... 

5.5%	 <unk>
1.3%	 grand
0.9%	 jour
0.8%	 plus
0.7%	 deux
0.7%	 yeux
0.7%	 calife
0.5%	 sultan
0.5%	 voix
0.4%	 même
0.4%	 main
0.4%	 soir
0.4%	 peu
0.4%	 nuit
0.4%	 autres
0.4%	 vie
0.3%	 hommes
0.3%	 homme
0.3%	 tête
0.3%	 maison
0.3%	 père
0.3%	 temps
0.3%	 place
0.3%	 monde
0.3%	 ville
0.3%	 génie
0.3%	 petite
0.3%	 loup
0.3%	 bras
0.3%	 roi


Le mot < unk > sera presque toujours le plus probable. Écrire une fonction get_next_word, variante de show_next_word qui retourne le mot le plus fréquent. Si ce mot est < unk >, votre fonction doit retourner le duexième mot le plus probable.

In [50]:
def get_next_word(scores):
    prob=F.softmax(scores,dim=2)
    num_word_display = 2
    p=prob[-1].squeeze()
    p, word_idx = torch.topk(p, num_word_display)
#     print(p, word_idx)
    if word_idx[0] == 0:
        return idx2word[word_idx[1]]
    else:
        return idx2word[word_idx[0]]
    

In [51]:
next_word = get_next_word(scores)

Écrire un bout de code qui prédit une phrase entière à partir de mysentence.

Cette phrase sera considérée comme terminée sur le modèle prédit < eos > ou bien après 10 itérations maximum.

In [54]:
print(mysentence + '...')

i= 0
not_finished = True
while i < 10 and not_finished :

    minibatch_data = sentence2vector_librivox_fr(mysentence)
      
    minibatch_data = minibatch_data.to(device)
    h = torch.zeros(1, 1, hidden_size)
    h=h.to(device)
    scores , h = net( minibatch_data , h )
    
    next_word = get_next_word(scores)
    mysentence += ' ' + next_word
    print(mysentence + '...')
    not_finished = next_word != '<eos>'
    i += 1
    

on entendait vaguement au dehors les grand <eos>...
on entendait vaguement au dehors les grand <eos> et...
on entendait vaguement au dehors les grand <eos> et <eos>...
